In [2]:
# imports 

import re
import sqlite3
import emoji

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm import tqdm
from matplotlib.ticker import MaxNLocator
from datetime import datetime 

tqdm.pandas()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# paths 

path_to_db = '/Users/alexraudvee/Downloads/test.db'

path_to_conv = '/Users/alexraudvee/Desktop/TU:e/Q4Y1_data_challenge_1/conversations.json'

In [4]:
# connection to the database with sentiment scores

try:
    connection = sqlite3.connect(path_to_db)
    print('Succesfuly connected to the database')

    query = """
        SELECT name FROM sqlite_master  
        WHERE type='table';
        """

    cursor = connection.cursor()
    cursor.execute(query)
    list_of_tables = cursor.fetchall()

    print('---TABLE NAMES IN DB---')
    for table_name in list_of_tables:
        print('-----------------------')
        print(table_name[0])

    print('-----------------------')
    
except sqlite3.Error as e:
    print(e)


Succesfuly connected to the database
---TABLE NAMES IN DB---
-----------------------
users
-----------------------
tweets
-----------------------
tweets_geo
-----------------------
replies
-----------------------
retweets
-----------------------
quotes
-----------------------
hashtags
-----------------------
sqlite_sequence
-----------------------
symbols
-----------------------
user_mentions
-----------------------


In [5]:
# tweets table
def get_tweets(conn):
    query = """
        SELECT *
        FROM tweets
    """

    df = pd.read_sql(query, conn)
    return df

# users table
def get_users(conn):
    query = """
        SELECT *
        FROM users
    """

    df = pd.read_sql(query, conn)
    return df

# tweets geo table 
def get_tweets_geo(conn):
    query = """
        SELECT *
        FROM tweets_geo
    """

    df = pd.read_sql(query, conn)
    return df

# replies table
def get_replies(conn):
    query = """
        SELECT *
        FROM replies
    """

    df = pd.read_sql(query, conn)
    return df

# retweets table
def get_retweets(conn):
    query = """
        SELECT *
        FROM retweets
    """

    df = pd.read_sql(query, conn)
    return df

# quotes table
def get_quotes(conn):
    query = """
        SELECT *
        FROM quotes
    """

    df = pd.read_sql(query, conn)
    return df

# hastags table 
def get_hashtags(conn):
    query = """
        SELECT *
        FROM hashtags
    """

    df = pd.read_sql(query, conn)
    return df

# symbols table 
def get_symbols(conn):
    query = """
        SELECT *
        FROM symbols
    """

    df = pd.read_sql(query, conn)
    return df

# user mentions table 
def get_user_mentions(conn):
    query = """
        SELECT *
        FROM user_mentions
    """

    df = pd.read_sql(query, conn)
    return df

In [6]:
# construct the dataframes for further analysis 

df_tweets = get_tweets(conn=connection)
df_users = get_users(conn=connection)
df_replies = get_replies(conn=connection)
df_retweets = get_retweets(conn=connection)
df_quotes = get_quotes(conn=connection)
df_tweets_geo = get_tweets_geo(conn=connection)
df_user_metions = get_user_mentions(conn=connection)
df_symbols = get_symbols(conn=connection)
df_hastags = get_hashtags(conn=connection)

In [7]:
# function to convert the sentiment label in numeric value

def convert_lable(lable):
    
    if lable == 'neutral':
        return int(0)
    elif lable == 'negative':
        return int(-1)
    elif lable == 'positive':
        return int(1)
    else:
        return None

In [8]:
# add new column with numeric values

df_tweets['sentiment_numeric'] = df_tweets['sentiment_label'].apply(lambda row: convert_lable(row))

In [9]:
# construct the dataframe with conversations 

df_covs = pd.read_json(path_to_conv)

In [13]:
df_sentiment = df_tweets[['tweet_id', 'sentiment_label']] 

df_covs_sentiment = pd.merge(df_covs, df_sentiment, on='tweet_id')

In [18]:
df_covs_sentiment['respond_time'] = df_covs_sentiment['timestamp'].diff()

In [20]:
df_covs_sentiment['timestamp']

0         2019-05-22 12:20:36.540
1         2019-05-22 12:32:28.175
2         2019-05-22 12:48:25.404
3         2019-05-22 21:50:31.782
4         2019-05-22 22:04:33.425
                    ...          
1040225   2020-03-30 18:39:02.293
1040226   2020-03-30 18:26:49.145
1040227   2020-03-30 18:33:44.275
1040228   2020-03-30 18:36:54.350
1040229   2020-03-30 18:40:56.391
Name: timestamp, Length: 1040230, dtype: datetime64[ns]